# 🕒 Input Feature Drift Detection for TTE Data

In [ ]:

import pandas as pd

df_train = pd.read_csv("df_train_tte.csv")
df_current = pd.read_csv("df_current_tte.csv")
df_train.head()


In [ ]:

feature_cols = [col for col in df_train.columns if col not in ['duration', 'event']]
print("Input features for drift detection:", feature_cols)


In [ ]:

from evidently.report import Report
from evidently.metric_preset import DataDriftPreset

report = Report(metrics=[DataDriftPreset()])
report.run(reference_data=df_train[feature_cols], current_data=df_current[feature_cols])
report.show()


In [ ]:

import numpy as np
import mlflow
from scipy.stats import entropy

def calculate_psi(ref, curr, buckets=10):
    def get_dist(x):
        counts, _ = np.histogram(x, bins=buckets)
        return counts / len(x)
    return entropy(get_dist(ref), get_dist(curr))

with mlflow.start_run(run_name="TTE_Feature_Drift"):
    for col in feature_cols:
        psi = calculate_psi(df_train[col], df_current[col])
        mlflow.log_metric(f"psi_{col}", psi)
        print(f"{col}: PSI = {psi:.4f}")
